In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
cd kinetic_cancer_final/scripts/physiology_comparison/

In [ ]:
# Collect all the objective function values
obj_values = pd.DataFrame(columns=['enzymes_perturbed', 'obj_value'])
for max_enz_mod in [1, 2, 5, 10, 50, 100, 150, 200, 250, 500, 877]:
    # Load the solution
    sol = pd.read_csv('./nra_solutions_small_model_significant_transporters/NRA_solution_MUT_to_WT_MILP_new_REMI_data_small_model_{}_max_enz_mod.csv'.format(max_enz_mod), index_col=0)
    # Get the values of the auxilariy variables 
    aux_vars = [v for v in sol.index if v.startswith('AUXV_')]
    # Get the value of the objective function by adding the values of the auxiliary variables
    obj_value = sol.loc[aux_vars].sum().values[0]
    obj_values.loc[max_enz_mod] = obj_value

    # Find how many enzymes were perturbed
    epsilon = 1e-9
    active_enzyme_regulations = [v for v in sol.index if v.startswith('EU_') or v.startswith('ED_')]
    active_enzyme_regulations = [v for v in active_enzyme_regulations
                                    if np.abs(sol.loc[v].values[0]) > epsilon]
    print('Number of enzymes perturbed:', len(active_enzyme_regulations))
    print('Objective value:', obj_value, '\n')

    obj_values.loc[max_enz_mod] = [len(active_enzyme_regulations), obj_value]


# Find the REMI solutions
remi_sols = pd.DataFrame(columns=['enzymes_perturbed', 'obj_value'])
for remi_test in ['NRA_solution_MUT_to_WT_MILP_exact_REMI_ratios', 'NRA_solution_MUT_to_WT_MILP_lower_bound_REMI_ratios']:
    # Load the solution
    sol = pd.read_csv('./nra_solutions_small_model_significant_transporters/{}.csv'.format(remi_test), index_col=0)
    # Get the values of the auxilariy variables 
    aux_vars = [v for v in sol.index if v.startswith('AUXV_')]
    # Get the value of the objective function by adding the values of the auxiliary variables
    obj_value = sol.loc[aux_vars].sum().values[0]
    
    # Find how many enzymes were perturbed
    epsilon = 1e-9
    active_enzyme_regulations = [v for v in sol.index if v.startswith('EU_') or v.startswith('ED_')]
    active_enzyme_regulations = [v for v in active_enzyme_regulations
                                    if np.abs(sol.loc[v].values[0]) > epsilon]
    print('Number of enzymes perturbed:', len(active_enzyme_regulations))
    print('Objective value:', obj_value, '\n')


    remi_sols.loc[remi_test] = [len(active_enzyme_regulations), obj_value]


In [ ]:
original_value = 371.2092331511442

In [ ]:
# Make a figure with the final values of the objective function vs the maximum number of enzyme modifications
plt.figure(figsize=(12, 8))
plt.plot(obj_values.enzymes_perturbed, obj_values.obj_value, marker='o')

# Add a line for the original value of the objective function with a different color
plt.axhline(original_value, color='red', linestyle='--', label='Original value')

# Add the REMI solutions as dashed lines with separate colors
# plt.axhline(remi_sols.loc['NRA_solution_MUT_to_WT_MILP_exact_REMI_ratios'].obj_value, color='blue', linestyle='--', label='REMI exact ratios')
# plt.axhline(remi_sols.loc['NRA_solution_MUT_to_WT_MILP_lower_bound_REMI_ratios'].obj_value, color='green', linestyle='--', label='REMI ratios imposed')

# Set font sizes
plt.xlabel('Enzymes perturbed', fontsize=16)
plt.ylabel('Sum of absolute flux deviations', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=14)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))

# Plot the main curve
plt.plot(
    obj_values.enzymes_perturbed,
    obj_values.obj_value,
    marker='o',
    color='steelblue',
    linewidth=2,
    markersize=6,
    label='Difference after enzyme changes'
)

# Add the reference (original value)
plt.axhline(
    original_value,
    color='firebrick',
    linestyle='--',
    linewidth=2,
    label='Original Difference (no changes)'
)

# Labels for a biology audience
plt.xlabel('Number of enzyme activity changes', fontsize=16)
plt.ylabel('Difference in metabolic activity', fontsize=16)

# Axis styling
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Legend
plt.legend(fontsize=14, frameon=False, loc='center right')

# Clean aesthetics
plt.tight_layout()
plt.savefig(f'../../results/physiology_comparison/difference_enzyme_changes.pdf',
            transparent=True,
            bbox_inches='tight',
            pad_inches=0.1,
            dpi=300)
plt.show()
